In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import yfinance as yf
import pandas_ta as ta
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
tickers ={'jpy': 'JPY%3DX',
        'eur': 'EURUSD%3DX',
        'gbp': 'GBPUSD%3DX',
        'franc': 'CHFUSD%3DX',
        'us100': 'NQ%3DF',
        'us30': 'YM%3DF',
        'us500': 'ES%3DF',
        'crude oil': 'CL%3DF',
        'gold': 'GC%3DF',
        'bitcoin': 'BTC-USD',
        'Apple': 'AAPL',
        'Microsoft': 'MSFT',
        'Google': 'GOOGL',
        'Amazon': 'AMZN',
        'NVIDIA': 'NVDA',
        'Berkshire': 'BRK-B',
        'Meta': 'META',
        'Tesla': 'TSLA'

    }
#import charts as cr
#cr.stoc('Apple',21)

def get_asset_df(asset_name):
    # asset_name=lower(asset_name)
    asset_map = {
        'jpy': 'JPY%3DX',
        'eur': 'EURUSD%3DX',
        'gbp': 'GBPUSD%3DX',
        'franc': 'CHFUSD%3DX',
        'us100': 'NQ%3DF',
        'us30': 'YM%3DF',
        'us500': 'ES%3DF',
        'crude oil': 'CL%3DF',
        'gold': 'GC%3DF',
        'bitcoin': 'BTC-USD',
        'Apple': 'AAPL',
        'Microsoft': 'MSFT',
        'Google': 'GOOGL',
        'Amazon': 'AMZN',
        'NVIDIA': 'NVDA',
        'Berkshire': 'BRK-B',
        'Meta': 'META',
        'Tesla': 'TSLA',
        'UGI': 'UNH',
        'Exxon': 'XOM',
        'Johnson&Johnson': 'JNJ',
        'Visa': 'V',
        'Procter&Gamble': 'PG',
        'JPMorgan': 'JPM',
        'Mastercard': 'MA',
        'EliLilly': 'LLY',
        'Chevron': 'CVX',
        'HomeDepot': 'HD',
        'Pfizer': 'PFE',
        'AbbVie': 'ABBV',
        'Merck': 'MRK',
        'Pepsi.': 'PEP',
        'Coca-Cola': 'KO',
        'Broadcom': 'AVGO',
        'Costco': 'COST',
        'WaltDisney': 'DIS',
        'Comcast': 'CMCSA',
        'Cisco': 'CSCO',
        'Intel': 'INTC',
        'Salesforce': 'CRM',
        'TexasInstruments ': 'TXN',
        'ThermoFisher ': 'TMO',
        'Bristol-Myers Squibb': 'BMY',
        'Verizon': 'VZ',
        'Nike': 'NKE',
        'Oracle': 'ORCL',
        'NextEra Energy': 'NEE',
        'McDonald ': 'MCD',
        'Adobe': 'ADBE',
        'PhilipMorris ': 'PM',
        'AT&T': 'T',
        'AbbottLaboratories': 'ABT',
        'AdvancedMicro': 'AMD',
        'UnionPacific': 'UNP',
        'Medtronic': 'MDT',
        'CVSHealth': 'CVS',
        'Qualcomm': 'QCOM',
        'Honeywell': 'HON',
        'GoldmanSachs ': 'GS'
    }
    
    if asset_name not in asset_map:
        raise ValueError(f"Asset '{asset_name}' not recognized. Available options are: {', '.join(asset_map.keys())}")
    
    ticker = asset_map[asset_name]
    
    df = yf.download(ticker, start=(datetime.now()-timedelta(days=31)), end=(datetime.now()+timedelta(days=1)), interval='1d')
    
    return df


In [2]:
trend_reversals = []
for ticker in tickers.keys():
    trend_reversals_dict={}
    df=get_asset_df(ticker)
    df['Week'] = df.index.to_period('W')


    weekly_data = df.groupby('Week').agg(
        High=('High', 'max'),
        Low=('Low', 'min'),
        Close=('Close', 'last')
    )

    weekly_data['Pivot'] = (weekly_data['High'] + weekly_data['Low'] + weekly_data['Close']) / 3
    weekly_data['R1'] = (2 * weekly_data['Pivot']) - weekly_data['Low']
    weekly_data['S1'] = (2 * weekly_data['Pivot']) - weekly_data['High']


    weekly_data[['Pivot', 'R1', 'S1']] = weekly_data[['Pivot', 'R1', 'S1']].shift(1)
    df = df.merge(weekly_data[['Pivot', 'R1', 'S1']], how='left', left_on='Week', right_index=True)


    df = df.dropna(subset=['Pivot'])

    df['Trend'] = df.apply(
    lambda row: 'Bullish' if row['Close'] > row['Pivot'] else 'Bearish',
    axis=1
    )
    trend=df['Trend'].iloc[-1]
    prior_trend=df['Trend'].iloc[-2]
    last_close_price=df['Close'].iloc[-1]
    pivot=df['Pivot'].iloc[-1]
    
    trend_reversals_dict={
        'ticker':ticker,
        'price':last_close_price,
        'pivot_price':pivot,
        'prior_trend':prior_trend,
        'trend':trend
    }
    trend_reversals.append(trend_reversals_dict)



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [3]:
trend_reversals=pd.DataFrame(trend_reversals)


In [4]:
def conditional_round(value):
    if abs(value) < 10:
        return round(value, 4)  
    else:
        return round(value, 2)  

trend_reversals['price'] = trend_reversals['price'].apply(conditional_round)
trend_reversals['pivot_price'] = trend_reversals['pivot_price'].apply(conditional_round)

In [5]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

def send_email(receiver_email, subject, html_body):
    sender_email = "turalaliyev.142002@gmail.com"
    password = "lidi ylth vbrb anpi"

    # Set up the email content
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = receiver_email
    message['Subject'] = subject

    # Attach the HTML body
    message.attach(MIMEText(html_body, 'html'))

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message.as_string())
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")
    finally:
        server.quit()

# Convert DataFrame to HTML with conditional formatting
html_body = '''
<!DOCTYPE html>
<html>
<head>
    <style>
        table {
            width: 100%;
            border-collapse: collapse;
        }
        th, td {
            border: 1px solid black;
            padding: 8px;
            text-align: left;
        }
        th {
            background-color: #f2f2f2;
        }
        .row-bearish {
            background-color: #ffcccc; /* Light red for bearish rows */
        }
        .row-bullish {
            background-color: #ccffcc; /* Light green for bullish rows */
        }
        .prior-trend-bearish {
            background-color: #ffcccc; /* Light red for bearish prior_trend cells */
        }
        .prior-trend-bullish {
            background-color: #ccffcc; /* Light green for bullish prior_trend cells */
        }
    </style>
</head>
<body>
    <table>
        <thead>
            <tr>
                <th>ticker</th>
                <th>price</th>
                <th>pivot_price</th>
                <th>prior_trend</th>
                <th>trend</th>
            </tr>
        </thead>
        <tbody>
'''

# Loop through DataFrame rows and add conditional formatting
for _, row in trend_reversals.iterrows():
    trend_class = "row-bearish" if row['trend'] == "Bearish" else "row-bullish"
    prior_trend_class = "prior-trend-bearish" if row['prior_trend'] == "Bearish" else "prior-trend-bullish"
    html_body += f'''
            <tr class="{trend_class}">
                <td>{row['ticker']}</td>
                <td>{row['price']}</td>
                <td>{row['pivot_price']}</td>
                <td class="{prior_trend_class}">{row['prior_trend']}</td>
                <td>{row['trend']}</td>
            </tr>
    '''

html_body += '''
        </tbody>
    </table>
</body>
</html>
'''

# Send the email with the HTML table
subject = "Pivot price levels"
receiver_email = "aliyevtural142002@gmail.com"
send_email(receiver_email, subject, html_body)


Email sent successfully!
